# Modelling dam volumes using DEA waterbodies
# Section 02  : *Model Training*
**Products used:** 
[DE Africa Waterbodies](https://docs.digitalearthafrica.org/en/latest/data_specs/Waterbodies_specs.html), 
[Department of Water Affairs and Sanitation, South Africa Dam Level and Volume Data](https://www.dws.gov.za/Hydrology/Verified/hymain.aspx)

## Background

### Digital Twin (DT)
The CGIAR Digital Twin initiative creates dynamic virtual models that combine real-time data, AI, and simulations to improve decision-making. Its prototype for the Limpopo River Basin focuses on enhancing water resource management and conservation.


## Description
This notebook presents a workflow for predicting dam levels and volumes using water surface area data from DE Africa's Waterbodies product, integrating data preprocessing, feature extraction, and Gradient Boosting modeling. 

As part of the CGIAR Initiative on Digital Innovation, this work contributes to a prototype Digital Twin for the Limpopo River Basin, designed to support real-time decision-making in water management. The Digital Twin leverages AI-driven tools to visualize and simulate the impact of decisions on the basin's ecosystem. To enhance prediction reliability, the model includes a correction mechanism to address unrealistic large drops in dam volume estimates.


## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell. 

### Load packages
Import Python packages that are used for the analysis.

In [1]:
%matplotlib inline

import os
import pandas as pd
import numpy as np
import xarray as xr
import seaborn as sns
import datacube
import joblib

from scipy import interpolate
from scipy.optimize import curve_fit
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor

import matplotlib.pyplot as plt
import plotly.graph_objs as go
import plotly.express as px
import plotly.graph_objects as go

from deafrica_tools.waterbodies import get_waterbody, get_time_series, display_time_series
from IPython.display import Image

## Load Model Training Datasets


### Sample Data Overview

This dataset contains raw water levels data collected from DEA (Department of Environmental Affairs) in South Africa. The cell below reads this ancillary data necessary to conduct the volume prediction.

### Data Preparation and Feature Selection
This section involves selecting the relevant features and handling missing values.

In [2]:
merged_data = pd.read_csv("data/preprocess_data.csv")

In [3]:
features = merged_data[['calculated_level', 'water_area_ha']]
target = merged_data['Dam_Level']
print(f"Initial data shape: {features.shape}")
print(f"Missing values in features:\n{features.isnull().sum()}")

imputer = SimpleImputer(strategy='mean')
features_imputed = imputer.fit_transform(features)

features_imputed_df = pd.DataFrame(features_imputed, columns=features.columns)
print(f"Data shape after imputing missing values: {features_imputed_df.shape}")

Initial data shape: (159, 2)
Missing values in features:
calculated_level    0
water_area_ha       0
dtype: int64
Data shape after imputing missing values: (159, 2)


### Splitting Data for Training and Testing
Here, we split the data into training and test sets.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(features_imputed_df, target, test_size=0.2, random_state=42)
print(f"Training set size: {X_train.shape[0]} rows")
print(f"Test set size: {X_test.shape[0]} rows")


Training set size: 127 rows
Test set size: 32 rows


### Performing Cross-Validation
This step involves evaluating the model using cross-validation.

In [5]:
print("Performing cross-validation...")
gradient_boosting = GradientBoostingRegressor(random_state=42)
cv_scores = cross_val_score(gradient_boosting, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores).mean()
print(f"Cross-validated RMSE: {cv_rmse:.4f}")

Performing cross-validation...
Cross-validated RMSE: 0.7095


### Hyperparameter Tuning
We perform hyperparameter tuning using GridSearchCV to find the best combination of parameters for the Gradient Boosting Regressor.

In [6]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}
print("Performing hyperparameter tuning using GridSearchCV...")
grid_search = GridSearchCV(GradientBoostingRegressor(random_state=42), param_grid, cv=3, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
print(f"Best parameters: {grid_search.best_params_}")


Performing hyperparameter tuning using GridSearchCV...
Best parameters: {'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 300}


### Training the Best Model
We now train the model with the optimal hyperparameters found in the previous step.

In [7]:
print("\nTraining the best model with optimal hyperparameters...")
best_model.fit(X_train, y_train)
print("Model training completed.")


Training the best model with optimal hyperparameters...
Model training completed.


### Model Evaluation
We evaluate the trained model on the test dataset using RMSE, MAPE, and R² Score as performance metrics.

In [8]:
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"\nModel evaluation results:\n - RMSE: {rmse:.4f}\n - MAPE: {mape:.4f}\n - R² Score: {r2:.4f}")



Model evaluation results:
 - RMSE: 0.2615
 - MAPE: 0.0063
 - R² Score: 0.9164


### Feature Importances
We analyze the importance of each feature to understand how much each one contributed to the model's predictions.

In [9]:
print("\nFeature Importances:")
feature_importances = best_model.feature_importances_
for feature, importance in zip(features.columns, feature_importances):
    print(f"Feature: {feature}, Importance: {importance:.4f}")


Feature Importances:
Feature: calculated_level, Importance: 0.5550
Feature: water_area_ha, Importance: 0.4450


### Saving the Trained Model
We save the trained model for future use.

In [10]:
print("\nSaving the trained model...")
os.makedirs("trained_models", exist_ok=True)
model_path = "trained_models/gradient_boosting_model.pkl"
joblib.dump(best_model, model_path)
print(f"Trained model saved successfully at: {model_path}")
y_pred_full = best_model.predict(features_imputed_df)
prediction = pd.Series(y_pred_full)

#saving test and train data
prediction.to_csv("data/prediction_data.csv")
target.to_csv("data/test_data.csv")
print("test and train data is saved....")



Saving the trained model...
Trained model saved successfully at: trained_models/gradient_boosting_model.pkl
test and train data is saved....


------------
## Additional information

<b> License </b> The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0).

Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

<b> Contact </b> If you need assistance, please post a question on the [DE Africa Slack channel](https://digitalearthafrica.slack.com/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).

If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

<b> Compatible datacube version </b>

**References:**
- Garcia Andarcia, M., Dickens, C., Silva, P., Matheswaran, K., & Koo, J. (2024). Digital Twin for management of water resources in the Limpopo River Basin: a concept. Colombo, Sri Lanka: International Water Management Institute (IWMI). CGIAR Initiative on Digital Innovation. 4p.
- Gurusinghe, T., Muthuwatta, L., Matheswaran, K., & Dickens, C. (2024). Developing a foundational hydrological model for the Limpopo River Basin using the Soil and Water Assessment Tool Plus (SWAT+). Colombo, Sri Lanka: International Water Management Institute (IWMI). CGIAR Initiative on Digital Innovation. 14p.
- Leitão, P. C., Santos, F., Barreiros, D., Santos, H., Silva, P., Madushanka, T., Matheswaran, K., Mutuwatte, L., Vickneswaran, K., Retief, H., Dickens, C., Garcia Andarcia, M. (2024). Operational SWAT+ Model: Advancing Seasonal Forecasting in the Limpopo River Basin. Colombo, Sri Lanka:  International Water Management Institute (IWMI). CGIAR Initiative on Digital Innovation.
- Mallick, Archita & Ghosh, Surajit & De Sarkar, Kounik & Roy, Sudip. (2024). Reservoir Water Level Forecasting using Deep Learning Technique. Conference: 2024 IEEE India Geoscience and Remote Sensing Symposium

## Embed into project background: 
The CGIAR Digital Innovation Initiative accelerates the transformation towards sustainable and inclusive agrifood systems by generating research-based evidence and innovative digital solutions. It is one of 32 initiatives of CGIAR, a global research partnership for a food-secure future, dedicated to transforming food, land, and water systems in a climate crisis.

### Contributors

**Hugo Retief**  
*Researcher*  
Email: [hugo@award.org.za](mailto:hugo@award.org.za)  

**Victoria Neema**  
*Earth Observation Scientist*  
Email: [victoria.neema@digitalearthafrica.org](mailto:victoria.neema@digitalearthafrica.org)  

**Kayathri Vigneswaran**  
*Junior Data Scientist*  
Email: [v.kayathri@cgiar.org](mailto:v.kayathri@cgiar.org)  


In [11]:
print(datacube.__version__)

1.8.20


**Last Tested:**

In [12]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2025-04-16'